In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
half = sym.Rational(1,2)

p = RollRig.base_params()

In [3]:
p

In [4]:
wing_frame = mf.HomogenousTransform().R_x(p.q[0]).R_y(p.alpha_r)
rhs_fwt_frame = wing_frame.Translate(0,p.s_w*half,0).R_x(-p.q[1])  # RHS Fwt Frame
lhs_fwt_frame = wing_frame.Translate(0,-p.s_w*half,0).R_x(p.q[2])  # RHS Fwt Frame

M_fwt = ele.MassMatrix(p.m_f,I_xx = p.I_xx_f)
M_w = ele.MassMatrix(p.m_w,I_xx = p.I_xx_w)
#M_lock = ele.MassMatrix(p.m_l)

bar_fwt_rhs = ele.RigidElement(rhs_fwt_frame.Translate(0,p.s_f*half,0),M_fwt,True)
bar_fwt_lhs = ele.RigidElement(lhs_fwt_frame.Translate(0,-p.s_f*half,0),M_fwt,True)
bar_wing = ele.RigidElement(wing_frame,M_w)

In [19]:
# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce_4.PerUnitSpan(p,wing_frame.Translate(0,p.y_w,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_r,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True).integrate((p.y_w,-p.s_w*half,p.s_w*half))

In [6]:
# Left FWT Aero Forces 
lhs_fwt_AeroForces = ef.AeroForce_4.PerUnitSpan(p,lhs_fwt_frame.Translate(0,-p.y_f,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_1,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True).integrate((p.y_f,0,p.s_f))

In [7]:
# Left FWT Aero Forces 
rhs_fwt_AeroForces = ef.AeroForce_4.PerUnitSpan(p,rhs_fwt_frame.Translate(0,p.y_f,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_2,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True).integrate((p.y_f,0,p.s_f))

In [20]:
wing_AeroForces.Q()

⎡           3        3    2        ⎤
⎢-V⋅a₀⋅c⋅ρ⋅s ⋅(1 - σ) ⋅cos (αᵣ)⋅q₀̇ ⎥
⎢──────────────────────────────────⎥
⎢                24                ⎥
⎢                                  ⎥
⎢                0                 ⎥
⎢                                  ⎥
⎣                0                 ⎦

In [21]:
Forcing = ef.CustomForce(None) 

CompositeForce = ef.CompositeForce([rhs_fwt_AeroForces,lhs_fwt_AeroForces,wing_AeroForces,Forcing])
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[bar_fwt_rhs,bar_fwt_lhs,bar_wing],ExtForces = CompositeForce)

In [27]:
sm.M.subs(p.alpha_r,0)

⎡       2  2                2  2                  2  2        2               
⎢  m_f⋅s ⋅σ ⋅cos(q₁)   m_f⋅s ⋅σ ⋅cos(q₂)   2⋅m_f⋅s ⋅σ    m_f⋅s ⋅σ⋅cos(q₁)   m_
⎢- ───────────────── - ───────────────── + ─────────── + ──────────────── + ──
⎢          4                   4                3               4             
⎢                                                                             
⎢                                                    2  2                2  2 
⎢                                               m_f⋅s ⋅σ ⋅cos(q₁)   m_f⋅s ⋅σ  
⎢                                               ───────────────── - ───────── 
⎢                                                       8               12    
⎢                                                                             
⎢                                                     2  2                2  2
⎢                                                m_f⋅s ⋅σ ⋅cos(q₂)   m_f⋅s ⋅σ 
⎢                                              - ───

In [28]:
sm.f.subs(p.alpha_r,0)

⎡                                                                             
⎢        ⎛      ⎛sin(q₀)⋅sin(q₁)   cos(q₀)⋅cos(q₁)⎞     ⎛1   σ⎞        ⎞      
⎢- g⋅m_f⋅⎜- s⋅σ⋅⎜─────────────── + ───────────────⎟ - s⋅⎜─ - ─⎟⋅cos(q₀)⎟ - g⋅m
⎢        ⎝      ⎝       4                 4       ⎠     ⎝2   2⎠        ⎠      
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                           g⋅
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [12]:
mf.LineariseMatrix(sm.M.subs(p.alpha_r,0),p.x,[0,0,0,0,0,0])

⎡     2  2        2          2        2  2        2          2       2  2     
⎢m_f⋅s ⋅σ    m_f⋅s ⋅σ   m_f⋅s    m_w⋅s ⋅σ    m_w⋅s ⋅σ   m_w⋅s   m_f⋅s ⋅σ    m_
⎢───────── - ──────── + ────── + ───────── - ──────── + ──────  ───────── - ──
⎢    6          2         2          12         6         12        24        
⎢                                                                             
⎢                         2  2        2                                   2  2
⎢                    m_f⋅s ⋅σ    m_f⋅s ⋅σ                            m_f⋅s ⋅σ 
⎢                    ───────── - ────────                            ─────────
⎢                        24         8                                    12   
⎢                                                                             
⎢                          2  2        2                                      
⎢                     m_f⋅s ⋅σ    m_f⋅s ⋅σ                                    
⎢                   - ───────── + ────────          

In [17]:
mf.LineariseMatrix(sm.f.subs(p.alpha_r,0),p.x[2:],[0,0,0,0])

⎡g⋅m_f⋅s⋅σ⋅q₁⋅sin(q₀)   g⋅m_f⋅s⋅σ⋅q₂⋅sin(q₀)         ⎛  s⋅σ⋅cos(q₀)     ⎛1   σ
⎢──────────────────── + ──────────────────── - g⋅m_f⋅⎜- ─────────── - s⋅⎜─ - ─
⎢         4                      4                   ⎝       4          ⎝2   2
⎢                                                                             
⎢                                               ⎛                           2 
⎢                           g⋅m_f⋅s⋅σ⋅cos(q₀)   ⎜  g⋅m_f⋅s⋅σ⋅sin(q₀)   m_f⋅s ⋅
⎢                         - ───────────────── + ⎜- ───────────────── - ───────
⎢                                   4           ⎝          4                  
⎢                                                                             
⎢                                                ⎛                         2 ⎛
⎢                            g⋅m_f⋅s⋅σ⋅cos(q₀)   ⎜g⋅m_f⋅s⋅σ⋅sin(q₀)   m_f⋅s ⋅⎝
⎢                          - ───────────────── + ⎜───────────────── - ────────
⎣                                    4           ⎝  

In [14]:
_Q = sm.ExtForces.Q().subs({p.alpha_r:0,p.alpha_1:sym.atan(sym.sin(p.q[2])*sym.sin(p.Lambda)),
                           p.alpha_2:sym.atan(sym.sin(p.q[1])*sym.sin(p.Lambda)),p.w_g:0})
test = mf.LineariseMatrix(_Q,p.x[2:],[0,0,0,0])

In [15]:
test.jacobian(p.qd)

⎡                    2  2 ⎛V⋅a₀⋅c⋅ρ⋅s⋅σ   V⋅a₀⋅c⋅ρ⋅s⎞                         
⎢            3  3   s ⋅σ ⋅⎜──────────── - ──────────⎟       ⎛            2  2 
⎢  V⋅a₀⋅c⋅ρ⋅s ⋅σ          ⎝     4             4     ⎠       ⎜  V⋅a₀⋅c⋅ρ⋅s ⋅σ  
⎢- ────────────── + ───────────────────────────────── + s⋅σ⋅⎜- ────────────── 
⎢        24                         2                       ⎝        8        
⎢                                                                             
⎢                                             2  2 ⎛  V⋅a₀⋅c⋅ρ⋅s⋅σ   V⋅a₀⋅c⋅ρ⋅
⎢                                     3  3   s ⋅σ ⋅⎜- ──────────── + ─────────
⎢                           V⋅a₀⋅c⋅ρ⋅s ⋅σ          ⎝       8             8    
⎢                           ────────────── + ─────────────────────────────────
⎢                                 48                          4               
⎢                                                                             
⎢                                               2  2

In [16]:
mf.LineariseMatrix(sm.f.subs(p.alpha_r,0),p.x[2:],[0,0,0,0])

⎡g⋅m_f⋅s⋅σ⋅q₁⋅sin(q₀)   g⋅m_f⋅s⋅σ⋅q₂⋅sin(q₀)         ⎛  s⋅σ⋅cos(q₀)     ⎛1   σ
⎢──────────────────── + ──────────────────── - g⋅m_f⋅⎜- ─────────── - s⋅⎜─ - ─
⎢         4                      4                   ⎝       4          ⎝2   2
⎢                                                                             
⎢                                               ⎛                           2 
⎢                           g⋅m_f⋅s⋅σ⋅cos(q₀)   ⎜  g⋅m_f⋅s⋅σ⋅sin(q₀)   m_f⋅s ⋅
⎢                         - ───────────────── + ⎜- ───────────────── - ───────
⎢                                   4           ⎝          4                  
⎢                                                                             
⎢                                                ⎛                         2 ⎛
⎢                            g⋅m_f⋅s⋅σ⋅cos(q₀)   ⎜g⋅m_f⋅s⋅σ⋅sin(q₀)   m_f⋅s ⋅⎝
⎢                          - ───────────────── + ⎜───────────────── - ────────
⎣                                    4           ⎝  

In [32]:
mf.LineariseMatrix(sm.f.subs(p.alpha_r,0),p.x[2:],[0,0,0,0])

⎡g⋅m_f⋅s⋅σ⋅q₁⋅sin(q₀)   g⋅m_f⋅s⋅σ⋅q₂⋅sin(q₀)         ⎛  s⋅σ⋅cos(q₀)     ⎛1   σ
⎢──────────────────── + ──────────────────── - g⋅m_f⋅⎜- ─────────── - s⋅⎜─ - ─
⎢         4                      4                   ⎝       4          ⎝2   2
⎢                                                                             
⎢                                               ⎛                           2 
⎢                           g⋅m_f⋅s⋅σ⋅cos(q₀)   ⎜  g⋅m_f⋅s⋅σ⋅sin(q₀)   m_f⋅s ⋅
⎢                         - ───────────────── + ⎜- ───────────────── - ───────
⎢                                   4           ⎝          4                  
⎢                                                                             
⎢                                                ⎛                         2 ⎛
⎢                            g⋅m_f⋅s⋅σ⋅cos(q₀)   ⎜g⋅m_f⋅s⋅σ⋅sin(q₀)   m_f⋅s ⋅⎝
⎢                          - ───────────────── + ⎜───────────────── - ────────
⎣                                    4           ⎝  

In [30]:
sm.f.subs(p.alpha_r,0)

⎡                                                                             
⎢        ⎛      ⎛sin(q₀)⋅sin(q₁)   cos(q₀)⋅cos(q₁)⎞     ⎛1   σ⎞        ⎞      
⎢- g⋅m_f⋅⎜- s⋅σ⋅⎜─────────────── + ───────────────⎟ - s⋅⎜─ - ─⎟⋅cos(q₀)⎟ - g⋅m
⎢        ⎝      ⎝       4                 4       ⎠     ⎝2   2⎠        ⎠      
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                           g⋅
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [44]:
sym.simplify(mf.LineariseMatrix(sm.M.subs(p.alpha_r,0),p.x[2:],[0,0,0,0]))

⎡ 2 ⎛       2                          2                ⎞       2             
⎢s ⋅⎝2⋅m_f⋅σ  - 6⋅m_f⋅σ + 6⋅m_f + m_w⋅σ  - 2⋅m_w⋅σ + m_w⎠  m_f⋅s ⋅σ⋅(σ - 3)  m
⎢────────────────────────────────────────────────────────  ────────────────  ─
⎢                           12                                    24          
⎢                                                                             
⎢                         2                                        2  2       
⎢                    m_f⋅s ⋅σ⋅(σ - 3)                         m_f⋅s ⋅σ        
⎢                    ────────────────                         ─────────       
⎢                           24                                    12          
⎢                                                                             
⎢                         2                                                   
⎢                    m_f⋅s ⋅σ⋅(3 - σ)                                         
⎢                    ────────────────               

In [41]:
_Q = sm.ExtForces.Q().subs({p.alpha_r:0,p.alpha_1:sym.atan(sym.sin(p.q[2])*sym.sin(p.Lambda)),
                           p.alpha_2:sym.atan(sym.sin(p.q[1])*sym.sin(p.Lambda)),p.w_g:0})
test = mf.LineariseMatrix(_Q,p.x[2:],[0,0,0,0])
sym.simplify(test.jacobian(p.qd)*(1/(p.V**2*p.rho)))

⎡           3              3  2                3  2        ⎤
⎢    -a₀⋅c⋅s         a₀⋅c⋅s ⋅σ ⋅(3 - σ)  a₀⋅c⋅s ⋅σ ⋅(σ - 3)⎥
⎢    ─────────       ──────────────────  ──────────────────⎥
⎢       24⋅V                96⋅V                96⋅V       ⎥
⎢                                                          ⎥
⎢      3  2                    3  3                        ⎥
⎢a₀⋅c⋅s ⋅σ ⋅(3 - σ)     -a₀⋅c⋅s ⋅σ                         ⎥
⎢──────────────────     ────────────             0         ⎥
⎢       96⋅V                48⋅V                           ⎥
⎢                                                          ⎥
⎢      3  2                                        3  3    ⎥
⎢a₀⋅c⋅s ⋅σ ⋅(σ - 3)                         -a₀⋅c⋅s ⋅σ     ⎥
⎢──────────────────          0              ────────────   ⎥
⎣       96⋅V                                    48⋅V       ⎦